In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from nltk.translate.bleu_score import sentence_bleu
from transformers import DistilBertModel,DistilBertTokenizer
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm

In [5]:
PATH_ROOT = "/home/shaul/workspace/GitHub/SOTA/"
%cd {PATH_ROOT}

/home/shaul/workspace/GitHub/SOTA


In [6]:
df = pd.read_csv("data/combined/combined_dataset.csv").drop(['Unnamed: 0'], axis = 1)

In [7]:
df.head(3)

,annotator,text1,text2,label,dataset,random,duration,total_seconds,pair_id,reduced_label
0,A3BCGN11HDM8QR,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",2,bible_human,0,0 days 00:00:12.000000000,12,pair_0,-1
1,A3SQ00HYQN7FYB,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",3,bible_human,0,0 days 00:00:12.000000000,12,pair_0,0
2,A5WAWW70PYRP,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",4,bible_human,0,0 days 00:07:19.000000000,439,pair_0,1


In [1]:
# # Removing Outliers
# 
# Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling, we want to currently:
# 
# 1. Find out who the bad actors are
# 2. See if there is any overlap from the dif. category of bad actors
# 
# (Removing bad phrases should also be done - but is beyond the scope of this notebook)
# 
# 
# The categories of bad actors are:
# 
# [x] - Slow Annotators - ba_time </br>
# [x] - Greater variance in random datasets than non-random datasets - ba_unvar_annotations </br>
# [x] - Unpopular (disagree with two others often) - ba_unpopular </br>
# [ ] - Inconsistent with sentiment != semantics </br>

# # Removing Outliers
# 
# Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
[ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics </br>

# Removing Outliers
 
Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
 [ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics

# Removing Outliers
 
Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
 [ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics </br>

# Removing Outliers

Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling, we want to currently:

1. Find out who the bad actors are
2. See if there is any overlap from the dif. category of bad actors

(Removing bad phrases should also be done - but is beyond the scope of this notebook)


The categories of bad actors are:

[x] - Slow Annotators </br>
[x] - Greater variance in random datasets than non-random datasets </br>
[ ] - Unpopular (disagree with two others often) </br>
[ ] - Inconsistent with sentiment != semantics </br>

## Time Outliers

Under the assumption that anyone that takes over the 95 percentile of time.

In [22]:
print(df.total_seconds.describe(percentiles = [.25,.5,.75,.9,.95]))

# ba = bad actor
ba_time = list(df[df.total_seconds > 336].annotator)

count    39660.000000
mean        86.010867
std        427.060791
min          1.000000
25%          7.000000
50%         12.000000
75%         28.000000
90%        169.000000
95%        336.000000
max      14079.000000
Name: total_seconds, dtype: float64


## Unvarianced Annotations
Labelers whos std is too low mean non-random - random difference is too high  

In [23]:
labelers = df[df.random==0].groupby(['annotator'])['label'].agg(['size','mean','std','min','max'])
labelers = labelers[labelers['size']>1]
#df = df[df.annotator.apply(lambda x:x in set(labelers.index))]

labelers_rand = df[df.random==1].groupby(['annotator'])['label'].agg(['size','mean','std','min','max'])
labelers_rand = labelers_rand[labelers_rand['size']>1]
labelers = labelers.join(labelers_rand, rsuffix = '_rand')
labelers['mean_random_gap'] = labelers['mean']-labelers['mean_rand']
labelers['std_ratio'] = labelers['std']/labelers['std_rand']

total_std = df.groupby('annotator')['label'].std()
total_std.name = 'total_std'
labelers = labelers.join(total_std)

In [24]:
ba_unvar_annotations = list(labelers[(labelers.total_std<1) & (labelers.mean_random_gap < 0)].index)

## Unpopular Annotators
Those who over 50% of the time, disagree with the other annotators (in the reduced label)

In [25]:
df_uniquelabels = df.groupby("pair_id")["reduced_label"].nunique()
pairs_twoagree = list(df_uniquelabels[(df.groupby("pair_id")["reduced_label"].nunique() == 2).values].index)
df_twoagree = df[df["pair_id"].isin(pairs_twoagree)]

df_id_reducedlabel = df_twoagree.groupby("pair_id")['reduced_label'].median()
df_twoagree['generally_accepted_label']  = df_id_reducedlabel.values.repeat(3)

In [26]:
df_unpopularopinion = df_twoagree[df_twoagree.reduced_label != df_twoagree.generally_accepted_label].groupby('annotator').size().reset_index()
df_unpopularopinion.columns = ['annotator','unpopular_opinion']

df_allopinions = df[df['annotator'].isin(list(df_unpopularopinion.annotator))].groupby('annotator').size().reset_index()
df_allopinions.columns = ['annotator','all_opinion']

df_opinion_all_unpop = df_allopinions.merge(df_unpopularopinion,on="annotator")

In [27]:
ba_unpopular = list(df_opinion_all_unpop[((df_opinion_all_unpop.unpopular_opinion / df_opinion_all_unpop.all_opinion) > 0.5) & (df_opinion_all_unpop.all_opinion > 4)].annotator)

## Sentiment // Semantic Understanding

In [29]:
# Sentiment analysis pipeline
sentiment_pipe = pipeline("sentiment-analysis")

RuntimeError: cuda runtime error (100) : no CUDA-capable device is detected at /pytorch/aten/src/THC/THCGeneral.cpp:47

In [ ]:
from tqdm import tqdm

text1_sent,text2_sent =[], []

pbar = tqdm(total = len(df)//100+1, position = 0, leave = True)
for i in range (len(df)//100+1):
    t1_s = sentiment_pipe(df.text1.tolist()[100*i:np.min([100*i+100,len(df)])])
    t2_s = sentiment_pipe(df.text2.tolist()[100*i:np.min([100*i+100,len(df)])])
    text1_sent+=t1_s
    text2_sent+=t2_s
    pbar.update()
pbar.close()
len(text1_sent)

# Combining all the results

In [ ]:
filtered.groupby('random').describe().T.join(df.groupby('random').describe().T,rsuffix = '_original')

In [ ]:
num_annot = filtered.annotator.nunique()
tot_annot = df.annotator.nunique()
print(f'after preforming all filtering we are reduced to {num_annot} annotators, comparing to {tot_annot} annotators we started with ({np.round(100*num_annot/tot_annot,2)}%)\nnew dataset contains {filtered.shape[0]} labeled pairs ({np.round(filtered.shape[0]*100/df.shape[0],2)}%) of the {df.shape[0]} in the original dataset')